# Import des bibliothèques

In [45]:
#importations de bibliothèques 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# chargement des deux tables de la base de données
orders_distance_stores_softmax = pd.read_csv("orders_distance_stores_softmax.csv")
orders_products__prior_specials = pd.read_csv("order_products__prior_specials.csv")

In [ ]:
orders_distance_stores_softmax

In [ ]:
orders_products__prior_specials

## Concaténation des deux tables

In [ ]:
# Obtenir l'ensemble des noms de colonnes uniques
colonnes_uniques = set(orders_distance_stores_softmax.columns).union(set(orders_products__prior_specials.columns))

Pour la concaténation, nous avons utilisé la méthode "merge" en se basant sur l'attribut commun des deux tables "order_id". Nous avons fixé la méthode de jointure à "inner" pour obtenir uniquement les lignes qui ont des correspondances dans les deux tables. 

In [ ]:
#combiner les deux tables de la base de données
data = orders_products__prior_specials.merge(orders_distance_stores_softmax, on='order_id', how='inner', suffixes=('', '_dup'))


In [ ]:
data.shape

In [ ]:
data.head()

Notre ensemble de données contient 1172312 et 15 colonnes dont les colonnes "Unnamed". Nous allons les supprimer (non pertinents pour notre étude).

In [ ]:
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0_dup'])

In [ ]:
data.columns

Nous avons donc 13 attributs donc l'attribut à prédire "reordered". Nous allons afficher le nombre d'échantillons pour chaque classe de l'attribut. Cette information sera pertinente pour l'analyse des résultats de prédiction. En cas de déséquilibre, il faut choisir des métriques spécifiques comme F1.

In [ ]:
data['reordered'].value_counts()

## Réduction des données (5%)

Avant de réduire nos données, nous allons les pourcentages des valeurs manquantes pour optimiser notre nouvel sous-ensemble.

In [ ]:
# Calcul de la moyenne des valeurs manquantes pour chaque colonne
pourcentage_valeurs_manquantes = (data.isnull().mean() * 100).round(2)

# Convertir le résultat en DataFrame pour une meilleure présentation
tableau_pourcentage_valeurs_manquantes = pd.DataFrame(pourcentage_valeurs_manquantes, columns=['Pourcentage de valeurs manquantes'])

tableau_pourcentage_valeurs_manquantes

Puisque nous avons plus de 1 milions de lignes (échantillons), nous nous permettons de supprimer les lignes où la valeur 'days_since_prior_order' est manquante.

In [ ]:
# Suppression des lignes où 'days_since_prior_order' est null
data = data.dropna(subset=['days_since_prior_order'])

Pour générer un sous ensemble de 5% des données, nous avons utilisé la méthode "train_test_split". Nous avons fait une stratification par rapport au "user_id" parce que notre prédiction de l'attribut "reordered" concerne aussi l'utilisateur (s'il a refait la même commande) donc nous avons voulu assurer qu'il y a un nombre équilibré et varié des utilisateurs (et leurs commandes).

In [ ]:
# stratification avec les IDs des utilisateurs
_, sampled_df = train_test_split(data, test_size=0.05, stratify=data['user_id'], random_state=42)

# Sous ensemble obtenu
print(sampled_df.columns)

In [ ]:
sampled_df.shape

Notre sous-ensemble contient 57923 échantillons et 12 prédicteurs.

In [ ]:
#afficher quelques statistiques sur les attributs du sous ensemble obtenu
sampled_stats = sampled_df.describe()
print(sampled_stats)

Nous allons vérifier qu'il y a un nombre raisonnable des commandes par utilisateur.

In [ ]:
# Calcul de la taille de chaque groupe d'user_id
group_sizes = sampled_df.groupby('user_id').size()

# Tentative d'afficher tous les résultats
# print(group_sizes.to_string())

In [ ]:
user_id = group_sizes.index.to_numpy()
user_id = np.reshape(user_id, (user_id.shape[0], 1))
values = group_sizes.to_numpy()
values = np.reshape(values, (values.shape[0], 1))
distributions = pd.DataFrame(np.hstack((user_id, values)), columns=["user_id", "values"])

In [13]:
distributions.plot.bar(x="user_id", y="values", 
                    xlabel="User", xticks=[], 
                    title="Distributions de la representativité des utilisateurs",
                    legend=None,
                    )

NameError: name 'distributions' is not defined

## Préparation des données 

Encodage des variables catégorielles (eval_set)

In [ ]:
# Initialisation du LabelEncoder
label_encoder = LabelEncoder()

# Identification des colonnes catégorielles
variables_categorielles = sampled_df.select_dtypes(include=['object', 'category']).columns

# Boucle sur chaque colonne catégorielle pour appliquer le LabelEncoder directement sur la colonne originale
for col in variables_categorielles:
    # Application du LabelEncoder à la colonne et écrasement de la colonne originale avec la version encodée
    sampled_df[col] = label_encoder.fit_transform(sampled_df[col])

In [ ]:
sampled_df.head()

In [ ]:
sampled_df.nunique()

Nous remarquons que "eval_set" a une seule valeur.

In [ ]:
sampled_df['eval_set'].dtype

Nous allons convertir les valeurs de l'attribut 'order_hour_of_day' pour qu'elles aient la même unité que l'attribut 'days_since_prior_order'. Nous voulons que les deux attributs soient exprimés en jours.

In [ ]:
# Convertir les heures en jours (1 jour = 24 heures) pour 'order_hour_of_day'
sampled_df['order_hour_of_day'] = sampled_df['order_hour_of_day'] / 24

## Vérification des valeurs dupliquées

In [ ]:
duplicates_df = sampled_df[sampled_df.duplicated()]
duplicates_df_sorted = duplicates_df.sort_values(by=['order_id'])
print(f"Nombre de valeurs dupliquées dans nos données \n{duplicates_df_sorted.sum()}")

Dans cette partie, nous avons vérifié que nos échantillons ne soient pas dupliqués.

## Visualisation des données 

In [ ]:
# Sélection des colonnes numériques
colonnes_numeriques = sampled_df.select_dtypes(include=['float64', 'int64', 'int32']).columns

# Définition de la taille de la grille
n_cols = 3
n_rows = int(np.ceil(len(colonnes_numeriques) / n_cols))

# Création des figures de boxplot avec ajustement de la taille
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6))  # Ajustez la largeur et la hauteur selon vos préférences
axes = axes.flatten()

for i, col in enumerate(colonnes_numeriques):
    sns.boxplot(y=sampled_df[col], ax=axes[i])
    axes[i].set_title(f'Boxplot de {col}')
    axes[i].set_ylabel(col)

# Masquer les axes supplémentaires s'il y en a
for j in range(i + 1, n_rows * n_cols):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

Analyse des visualisations des boxplots : 

Boxplot de store_id : Les identifiants de magasin montrent une certaine variabilité, mais la distribution est centrée, ce qui indique que les commandes proviennent d'un groupe relativement cohérent de magasins.

Boxplot de order_number : Les numéros de commande semblent avoir une distribution étendue avec la médiane vers le bas de la gamme, ce qui pourrait indiquer que de nombreux utilisateurs sont dans les premières étapes de l'historique de commande.

Boxplot de days_since_prior_order : Cette variable a une médiane proche de 0 avec des valeurs étendues vers le haut, ce qui indique des variations dans le temps écoulé depuis la dernière commande.

Boxplot de product_id : Les identifiants de produit sont distribués sur une large gamme avec quelques valeurs aberrantes, ce qui indique une grande variété de produits commandés.

Boxplot de special : Cette variable a une médiane à 0 et quelques valeurs aberrantes, ce qui suggère que des offres spéciales ne sont appliquées que dans un nombre limité de cas.

Boxplot de distance : Il y a des valeurs aberrantes, ce qui indique que bien que la plupart des utilisateurs soient à une distance moyenne des magasins, certains sont nettement plus éloignés.

Boxplot de order_dow : La distribution semble régulière sur toute la semaine sans valeurs aberrantes, suggérant une tendance uniforme dans les jours de passation des commandes.

Boxplot de add_to_cart_order : Il y a une grande variété dans l'ordre d'ajout des produits au panier, avec de nombreuses valeurs aberrantes, ce qui pourrait indiquer que certains produits ont tendance à être ajoutés au panier avant d'autres.

Boxplot de user_id : Les identifiants d'utilisateur ont une large distribution, ce qui suggère une grande base d'utilisateurs.

Boxplot de eval_set : Le boxplot n'est pas informatif ici puisque l'attribut contient une seule valeur.

Boxplot de order_hour_of_day : La distribution des heures de commande est centrée avec quelques valeurs aberrantes, indiquant des moments privilégiés pour passer commande au cours de la journée.

## Feature Engineering 

Dans ce contexte de ce tp, vous souhaitez prédire si un produit sera réordonné (reordered étant la variable cible) dans une future commande en fonction de l'historique d'achat d'un utilisateur. Basé sur cette objectif, voici les attributs que nous pensons ne pas avoir de valeur ajoutée pour la prédiction et donc être des candidats à l'élimination :

- order_id : C'est un identifiant unique pour chaque commande, il n'aide pas directement à prédire le réordonnancement. 

- eval_set : Il s'agit simplement d'une variable technique indiquant l'appartenance des données à un ensemble d'entraînement, de validation ou de test et qui garde la même valeur pour toutes les échantillons de notre ensemble de données. Elle ne doit pas être incluse dans l'entraînement du modèle pour éviter la fuite de données.

- store_id : L'identifiant du magasin n'a pas de relation avec la probabilité de réordonner un produit, cet attribut est à écarter.

L'attribut "user_id" en soi n'est pas une caractéristique prédictive directe, mais il est essentiel pour grouper les données et générer des caractéristiques comportementales significatives au niveau de l'utilisateur qui peuvent grandement améliorer la performance de votre modèle de prédiction.

In [ ]:
sampled_df = sampled_df.drop(columns = ['store_id', 'order_id', 'eval_set'])

## Division des données en ensembles Train et Test

In [ ]:
X = sampled_df.drop('reordered', axis=1)  # Caractéristiques
y = sampled_df['reordered']  # Variable cible

# Séparation des données en ensemble d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.columns

In [ ]:
X_test.columns

In [ ]:
numeric_columns = X_train.select_dtypes(include=['float64', 'int64']).columns

In [ ]:
# Réglage des paramètres pour afficher 3 figures par ligne
n_cols = 3  # Nombre de colonnes par ligne dans la grille de subplots
n_rows = int(np.ceil(len(numeric_columns) / n_cols))  # Calcul du nombre de lignes nécessaires

# Création d'une figure et de subplots avec la grille spécifiée
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 5))

# Aplatir le tableau d'axes pour une itération facile si nécessaire
axes_flat = axes.flatten()

# Visualisation avec des histogrammes pour chaque colonne numérique
for i, col in enumerate(numeric_columns):
    sns.histplot(X_train[col], color='blue', label='Train', kde=True, ax=axes_flat[i])
    sns.histplot(X_test[col], color='orange', label='Test', kde=True, ax=axes_flat[i])
    axes_flat[i].set_title(f'Distribution de {col} pour train et test')
    axes_flat[i].legend()

# Si le nombre total de colonnes numériques n'est pas un multiple de 3, masquer les axes vides
for j in range(i+1, len(axes_flat)):
    axes_flat[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Création d'une figure avec la grille de subplots pour des courbes de densité
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, n_rows * 4))

# Parcourir chaque colonne numérique et tracer des courbes de densité pour les ensembles train et test
for i, col in enumerate(numeric_columns):
    row = i // n_cols
    col_index = i % n_cols
    ax = axes[row, col_index]

    # Tracer la courbe de densité pour l'ensemble d'entraînement
    sns.kdeplot(data=X_train, x=col, fill=True, ax=ax, label='Train', color='blue', alpha=0.5)

    # Tracer la courbe de densité pour l'ensemble de test
    sns.kdeplot(data=X_test, x=col, fill=True, ax=ax, label='Test', color='orange', alpha=0.5)

    ax.set_title(f'Distribution de {col}')
    ax.legend()

# Cacher les subplots vides s'il y en a
for j in range(i+1, n_rows*n_cols):
    axes.flat[j].set_visible(False)

plt.tight_layout()
plt.show()

Les courbes montrent que les ensembles d'entraînement et de test ont des distributions cohérentes et comparables pour la majorité des attributs examinées. Cela signifie que la séparation des données a été bien réalisée, offrant ainsi une bonne représentativité et permettant d'anticiper que les modèles de machine learning entraînés sur l'ensemble d'entraînement devraient avoir des performances similaires sur l'ensemble de test. Les pics et creux correspondants dans les courbes de densité de product_id, add_to_cart_order, et days_since_prior_order reflètent des tendances spécifiques de comportement d'achat qui sont préservées entre les ensembles. Les caractéristiques périodiques comme order_dow et les pics d'heures dans order_hour_of_day sont particulièrement remarquables, révélant des schémas d'achat récurrents sur les jours et les heures qui pourraient être significatifs pour la prédiction du réordonnancement. En résumé, ces visualisations fournissent une confirmation visuelle que la séparation des données maintient l'intégrité statistique nécessaire pour une modélisation prédictive fiable.

# Sauvegarde des fichiers

In [ ]:
# Pour les données d'entraînement
X_train.to_csv('x_train.csv', index=False)
y_train.to_csv('y_train.csv', index=False)

# Pour les données de test
X_test.to_csv('x_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

# Pour sample_df
sampled_df.to_csv('final_data.csv', index=False)

# Entraînement des modèles (Question 2)

In [46]:
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(46338, 9) (46338, 1) (11585, 9) (11585, 1)


Utilisons la classe StandardScaler de sklearn pour normaliser les données, permettant ainsi de les mettre à la même échelle, ce qui améliorera les performances du modèle

In [47]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Implémentation du CNN


Puisque la couche de sortie utilise deux neurones, nous devons lui fournir un vecteur binaire à deux dimensions, c'est-à-dire contenant soit 0 ou 1. Cependant, notre vecteur cible est unidimensionnel. Par conséquent, nous allons utiliser un encodeur OneHotEncoder pour le transformer en un vecteur avec les dimensions appropriées.

In [48]:
from sklearn.preprocessing import OneHotEncoder

y_train_ = np.reshape(y_train, (m_train, 1))
y_test_ = np.reshape(y_test, (m_test, 1))

enc = OneHotEncoder(sparse_output=False)
y_train_ = enc.fit_transform(y_train_)
y_test_ = enc.fit_transform(y_test_)
print(y_train_.shape, y_test_.shape)

(46338, 2) (11585, 2)


Nous allons mettre en place un modèle de réseau de neurones convolutionnel (CNN) comprenant deux couches cachées utilisant la fonction d'activation relu, ainsi qu'une couche de sortie softmax composée de deux neurones, chacun représentant l'une des deux classes cibles.

In [49]:
import tensorflow as tf
from keras.layers import Dense, Input
from keras.models import Sequential

In [51]:
model = Sequential()

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

f1 = tf.keras.metrics.F1Score(average=None, threshold=None, name="f1_score", dtype=None)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1])
history = model.fit(x=X_train_scaled, y=y_train_, epochs=50)

Epoch 1/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.7940 - f1_score: 0.5995 - loss: 0.4479
Epoch 2/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8007 - f1_score: 0.6321 - loss: 0.4242
Epoch 3/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8074 - f1_score: 0.6366 - loss: 0.4181
Epoch 4/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8067 - f1_score: 0.6341 - loss: 0.4146
Epoch 5/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8055 - f1_score: 0.6352 - loss: 0.4187
Epoch 6/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8072 - f1_score: 0.6429 - loss: 0.4191
Epoch 7/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8080 - f1_score: 0.6400 - loss: 0.4150
Epoch 8/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8097 - f1_score: 0.6450 - loss: 0.4103
Epoch 9/50
1449/1449 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8085 - f1_score: 0.6492 - loss: 0.4159
Epoch 10/50
1449/1449 ━━━━━━━━━━━━━━━

In [ ]:
test_loss, test_acc = model.evaluate(x=X_test, y=y_test_)

In [ ]:
plt.plot(history.history['accuracy'], label="La précision d'entrainement.")
plt.title('Courbe d\'exactitude')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'], label="Perte d'entrainement")
plt.title('Courbe de perte')
plt.xlabel('Epoch')
plt.ylabel('Perte')
plt.legend()
plt.show()

### Implémentation SVM

Nous implémentons la méthode de classification Machine à vecteur de support, avec un gridsearch pour la recherche des meilleurs hyper-parametres (Le noyeau et la constante C).

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
y_train_ = np.reshape(y_train, (-1,))
y_test_ = np.reshape(y_test, (-1,))

print(y_train_.shape, y_test_.shape)

(46338,) (11585,)


Implémentation de la méthode SVM de la librairie sklearn, avec la recherche d'hyper-paramêtres 

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category = ConvergenceWarning)

parameters = {'kernel': ('poly', 'rbf'), 'C': [0.1, 1, 10]}
svc = svm.SVC(max_iter = 100)
clf = GridSearchCV(svc, parameters)
clf.fit(X_train_scaler, y_train_)

print(clf.best_params_)

(46338, 9) (46338,)
{'C': 0.1, 'kernel': 'poly'}


In [ ]:
y_pred = clf.predict(X_test_scaler)

print("\nRapport de classification :")
print(classification_report(y_test, y_pred))


Rapport de classification :
              precision    recall  f1-score   support

           0       0.20      0.19      0.19      2486
           1       0.78      0.79      0.79      9099

    accuracy                           0.66     11585
   macro avg       0.49      0.49      0.49     11585
weighted avg       0.66      0.66      0.66     11585

